In [1]:
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
import pandas as pd
import datetime as dt
import time
import random


In [2]:
pt = TrendReq()

In [3]:
def google_trend(keyword, date = dt.date(2014,1,5)):
    
    #Get daily values (by week)
    date_iter = date
    delta = dt.timedelta(7-1)
    day = dt.timedelta(1)

    daily_data = []

    while date_iter < dt.date.today():

        print(str(date_iter) + " " + str(date_iter+delta))

        pt.build_payload([keyword], timeframe = str(date_iter) + " " + str(date_iter+delta))
        
        try:
            data = pt.interest_over_time()
        except ResponseError as e:
            print("ResponseError.", e)

        daily_data.append(pt.interest_over_time())

        time.sleep(random.randint(1,7))
        
        date_iter += delta + day

    df_daily = pd.concat(daily_data)
    
    #Get daily values (by 70-day)
    date_iter = date
    delta = dt.timedelta(70-1)
    day = dt.timedelta(1)

    daily_data_check = []

    while date_iter < dt.date.today():

        pt.build_payload([keyword], timeframe = str(date_iter) + " " + str(date_iter+delta))
        
        try:
            data = pt.interest_over_time()
        except ResponseError as e:
            print("ResponseError.", e)

        daily_data_check.append(pt.interest_over_time())

        date_iter += delta + day
        
        time.sleep(random.randint(1,7))        

    df_daily_check = pd.concat(daily_data_check)  

    # Get weekly values
    print (str(date)+" "+str(dt.date.today()))
    
    pt.build_payload([keyword],timeframe = str(date)+" "+str(dt.date.today()))

    df_weekly = pt.interest_over_time()

    # Combine daily + weekly
    df = pd.concat([df_daily, df_daily_check, df_weekly], axis=1)

    df.drop(columns="isPartial", inplace=True)

    df.columns = ["daily","daily_check","weekly"]

    # Create adjusted value
    df["week_sum"] = df.daily.rolling(7).sum().shift(-6)
    df["week_sum"] = df.week_sum.mask(df.weekly.isna(),None)
    df["week_sum"] = df.week_sum.fillna(method="ffill")
    
    df["weekly"] = df.weekly.fillna(method="ffill")
    
    df["val"] = df.daily / df.week_sum * df.weekly
    
    df["val_check"] = df.val / df.daily_check

    # Output.csv
    df["val"].to_csv("crypto_csv/trend_"+keyword+".csv",header=True)
    
    return df

In [6]:
df = google_trend("blockchain")

2014-01-05 2014-01-11
2014-01-12 2014-01-18
2014-01-19 2014-01-25
2014-01-26 2014-02-01
2014-02-02 2014-02-08
2014-02-09 2014-02-15
2014-02-16 2014-02-22
2014-02-23 2014-03-01
2014-03-02 2014-03-08
2014-03-09 2014-03-15
2014-03-16 2014-03-22
2014-03-23 2014-03-29
2014-03-30 2014-04-05
2014-04-06 2014-04-12
2014-04-13 2014-04-19
2014-04-20 2014-04-26
2014-04-27 2014-05-03
2014-05-04 2014-05-10
2014-05-11 2014-05-17
2014-05-18 2014-05-24
2014-05-25 2014-05-31
2014-06-01 2014-06-07
2014-06-08 2014-06-14
2014-06-15 2014-06-21
2014-06-22 2014-06-28
2014-06-29 2014-07-05
2014-07-06 2014-07-12
2014-07-13 2014-07-19
2014-07-20 2014-07-26
2014-07-27 2014-08-02
2014-08-03 2014-08-09
2014-08-10 2014-08-16
2014-08-17 2014-08-23
2014-08-24 2014-08-30
2014-08-31 2014-09-06
2014-09-07 2014-09-13
2014-09-14 2014-09-20
2014-09-21 2014-09-27
2014-09-28 2014-10-04
2014-10-05 2014-10-11
2014-10-12 2014-10-18
2014-10-19 2014-10-25
2014-10-26 2014-11-01
2014-11-02 2014-11-08
2014-11-09 2014-11-15
2014-11-16

In [7]:
df

,daily,daily_check,weekly,week_sum,val,val_check
date,,,,,,
2014-01-05,64,63,3.0,501.0,0.383234,0.006083
2014-01-06,100,61,3.0,501.0,0.598802,0.009816
2014-01-07,72,38,3.0,501.0,0.431138,0.011346
2014-01-08,75,50,3.0,501.0,0.449102,0.008982
2014-01-09,63,67,3.0,501.0,0.377246,0.005631
2014-01-10,54,48,3.0,501.0,0.323353,0.006737
2014-01-11,73,43,3.0,501.0,0.437126,0.010166
2014-01-12,73,48,3.0,598.0,0.366221,0.007630
2014-01-13,93,56,3.0,598.0,0.466555,0.008331


In [4]:
df = google_trend("ethereum")

2014-01-05 2014-01-11
2014-01-12 2014-01-18
2014-01-19 2014-01-25
2014-01-26 2014-02-01
2014-02-02 2014-02-08
2014-02-09 2014-02-15
2014-02-16 2014-02-22
2014-02-23 2014-03-01
2014-03-02 2014-03-08
2014-03-09 2014-03-15
2014-03-16 2014-03-22
2014-03-23 2014-03-29
2014-03-30 2014-04-05
2014-04-06 2014-04-12
2014-04-13 2014-04-19
2014-04-20 2014-04-26
2014-04-27 2014-05-03
2014-05-04 2014-05-10
2014-05-11 2014-05-17
2014-05-18 2014-05-24
2014-05-25 2014-05-31
2014-06-01 2014-06-07
2014-06-08 2014-06-14
2014-06-15 2014-06-21
2014-06-22 2014-06-28
2014-06-29 2014-07-05
2014-07-06 2014-07-12
2014-07-13 2014-07-19
2014-07-20 2014-07-26
2014-07-27 2014-08-02
2014-08-03 2014-08-09
2014-08-10 2014-08-16
2014-08-17 2014-08-23
2014-08-24 2014-08-30
2014-08-31 2014-09-06
2014-09-07 2014-09-13
2014-09-14 2014-09-20
2014-09-21 2014-09-27
2014-09-28 2014-10-04
2014-10-05 2014-10-11
2014-10-12 2014-10-18
2014-10-19 2014-10-25
2014-10-26 2014-11-01
2014-11-02 2014-11-08
2014-11-09 2014-11-15
2014-11-16

In [5]:
df

,daily,daily_check,weekly,week_sum,val,val_check
date,,,,,,
2014-01-05,0,0,0.0,165.0,0.000000,NaN
2014-01-06,22,14,0.0,165.0,0.000000,0.000000
2014-01-07,0,0,0.0,165.0,0.000000,NaN
2014-01-08,0,7,0.0,165.0,0.000000,0.000000
2014-01-09,21,7,0.0,165.0,0.000000,0.000000
2014-01-10,22,14,0.0,165.0,0.000000,0.000000
2014-01-11,100,55,0.0,165.0,0.000000,0.000000
2014-01-12,58,0,0.0,401.0,0.000000,NaN
2014-01-13,33,13,0.0,401.0,0.000000,0.000000
